<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Parking_spot_detection_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parking Spot Detector 1


###Notice:
Since the model detects 3 different parking slots, it is difficult to detect them and if you want to continue this in the future, you must create a mask for each dataset and train on the images with some masks you created not the masks in the dataset, since they are a few and not accuracte enough,
 or you can combine this two projects to get a better project of them all.


In [ ]:
import os
import cv2
import json
import pickle
import sklearn
import skimage
import zipfile
import IPython
import scipy.io
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from sklearn import model_selection

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d blanderbuss/parking-lot-dataset
!unzip \*.zip && rm *.zip
IPython.display.clear_output()

##Image Classifier
At first we need to create a model to predict if a spot shown in the image is Empty or Occupied.

We use the [**parking lot dataset**](https://www.kaggle.com/datasets/blanderbuss/parking-lot-dataset) from kaggle for both classification and detection tasks.

###Data

In [ ]:
cls_data = './PKLot/PKLotSegmented'
train_path = './data/train'
val_path   = './data/validation'
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

VAL_THRESHOLD = 0.2

In [ ]:
#since the models are big so we need to select only a portion of the data so the ram won't get fulled
limit = 80
data_arr = []

for cat in os.listdir(cls_data):
  for crs in os.listdir(cls_data+'/'+cat):
    for day in os.listdir(cls_data+'/'+cat+'/'+crs):
      for E_O in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day):
        for img in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day+'/'+E_O)[:limit]:
          path = os.path.join(cls_data, cat, crs, day, E_O, img)
          folder_name = train_path if np.random.randn()<VAL_THRESHOLD else val_path
          target = os.path.join(folder_name, img)
          label = 0 if E_O == 'Empty' else 1
          data_arr.append([target, label])
          os.replace(path, target)


data = pd.DataFrame(data_arr, columns=['path', 'label'])
data

In [ ]:
data.groupby(['label']).count()

In [ ]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory('./data',
                                                               validation_split=0.2,
                                                               subset='both',
                                                               seed=32,
                                                               image_size=(150, 150),
                                                               batch_size=32)

### Visualization

In [ ]:
def show_images(ds, augmentation=False):
  plt.figure(figsize=(3, 3))
  for images, labels in ds.take(1):
      for i in range(9):
          ax = plt.subplot(3, 3, i + 1)
          if augmentation:
            images = data_augmentation(images)
            i = 0
          plt.imshow(images[i].numpy().astype("uint8"))
          plt.title(int(labels[i]))
          plt.axis("off")

In [ ]:
show_images(train_ds)

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(factor=0.15),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    tf.keras.layers.RandomFlip(),
    tf.keras.layers.RandomContrast(factor=0.1),
    ], name="img_augmentation")

In [ ]:
show_images(train_ds, augmentation=True)

In [ ]:
augmented_train_ds = train_ds.map(lambda img, label : (data_augmentation(img), label),
                                  num_parallel_calls=tf.data.AUTOTUNE)

#(buffered) Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
def plot_hist(hist, acc='binary_accuracy'):
    plt.plot(hist.history[f"{acc}"])
    plt.plot(hist.history[f"val_{acc}"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

### Model 1:
**Yolo V8**

Since the Yolo model needs some special data directory order, we need to change the data directory


In [ ]:
#install necessary libraries
!pip install awscli
!pip install ultralytics

import ultralytics
IPython.display.clear_output()

In [ ]:
cls_data = './PKLot/PKLotSegmented'
train_path_1 = './data/train/Empty/'
train_path_2 = './data/train/Occupied/'
val_path_1   = './data/validation/Empty/'
val_path_2   = './data/validation/Occupied/'

os.makedirs(train_path_1, exist_ok=True)
os.makedirs(val_path_1, exist_ok=True)
os.makedirs(train_path_2, exist_ok=True)
os.makedirs(val_path_2, exist_ok=True)

VAL_THRESHOLD = 0.2
data_arr = []
limit = 40


data = pd.DataFrame(data_arr, columns=['path', 'label'])
for cat in os.listdir(cls_data):
  for crs in os.listdir(cls_data+'/'+cat):
    for day in os.listdir(cls_data+'/'+cat+'/'+crs):
      for E_O in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day):
        for img in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day+'/'+E_O)[:limit]:
          path = os.path.join(cls_data, cat, crs, day, E_O, img)
          folder_name = 'train' if np.random.randn()<VAL_THRESHOLD else 'validation'
          target = os.path.join('./data', folder_name, E_O, img)
          label = 0 if E_O == 'Empty' else 1
          data_arr.append([target, label])
          os.replace(path, target)
          break

data = pd.DataFrame(data_arr, columns=['path', 'label'])
data

In [ ]:
# model = YOLO('yolov8n.yaml').load('yolov8n-cls.pt')  # build from YAML and transfer weights
model = ultralytics.YOLO('yolov8n-cls.pt')
model.train(data='./data', epochs = 10, device='cpu', seed=32)

### model 2:
SVC model from sklearn since our data is so simple that CNN can't detect any patterns we can get a simpler model like SVC to handle this problem.

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

for path, label in data_arr:
  folder = path.split('/')[2]
  img = skimage.io.imread(path)
  img = skimage.transform.resize(img, (20, 20))

  if folder == 'validation':
    x_test.append(img.flatten())
    y_test.append(label)
  else:
    x_train.append(img.flatten())
    y_train.append(label)


x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

In [ ]:
clf = svm.SVC()

parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

grid_search = model_selection.GridSearchCV(clf, parameters)

In [ ]:
grid_search.fit(x_train, y_train)

# test performance
best_estimator = grid_search.best_estimator_

y_prediction = best_estimator.predict(x_test)

score = metrics.accuracy_score(y_prediction, y_test)

print(f'{score * 100}% of samples were correctly classified')

pickle.dump(best_estimator, open('./drive/MyDrive/DATA/car_spot_model.p', 'wb'))

In [ ]:
test_data = []
test_label = []

for cat in os.listdir(cls_data):
  for crs in os.listdir(cls_data+'/'+cat):
    for day in os.listdir(cls_data+'/'+cat+'/'+crs):
      for E_O in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day):
        for img in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day+'/'+E_O):
          path = os.path.join(cls_data, cat, crs, day, E_O, img)
          folder_name = train_path if np.random.randn()<VAL_THRESHOLD else val_path
          target = os.path.join(folder_name, img)
          label = 0 if E_O == 'Empty' else 1
          test_data.append(path)
          test_label.append(label)


test = pd.DataFrame(test_data, columns=['path'])
test

In [ ]:
scores = []
for path, label in zip(test_data, test_label):
  folder = path.split('/')[2]
  img = skimage.io.imread(path)
  img = skimage.transform.resize(img, (20, 20))
  img = np.asarray(img.flatten())
  img = np.expand_dims(img, 0)
  pred = best_estimator.predict(img)
  score = metrics.accuracy_score(pred, [label])
  scores.append(score)

In [ ]:
test_data = []
test_label = []

for cat in os.listdir(cls_data):
  for crs in os.listdir(cls_data+'/'+cat):
    for day in os.listdir(cls_data+'/'+cat+'/'+crs):
      for E_O in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day):
        for img in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day+'/'+E_O):
          path = os.path.join(cls_data, cat, crs, day, E_O, img)
          folder_name = train_path if np.random.randn()<VAL_THRESHOLD else val_path
          target = os.path.join(folder_name, img)
          label = 0 if E_O == 'Empty' else 1
          test_data.append(path)
          test_label.append(label)


test = pd.DataFrame(test_data, columns=['path'])
test

In [ ]:
scores = []
for path, label in zip(test_data, test_label):
  folder = path.split('/')[2]
  img = skimage.io.imread(path)
  img = skimage.transform.resize(img, (20, 20))
  img = np.asarray(img.flatten())
  img = np.expand_dims(img, 0)
  pred = best_estimator.predict(img)
  score = metrics.accuracy_score(pred, [label])
  scores.append(score)

In [ ]:
totall_score = np.sum(scores) / len(scores)
print(f'{totall_score * 100:.4f}% of samples were correctly classified, from {len(scores)}')
print(f'In totall, The model got {int(np.sum(scores))} / {len(scores)} Right answer')
print(f'In totall, The model got {len(scores) - int(np.sum(scores))} / {len(scores)} Wrong answer')

## Detection

In [ ]:
cls_data = './PKLot/PKLotSegmented'
train_path_1 = './data/train/Empty/'
train_path_2 = './data/train/Occupied/'
val_path_1   = './data/validation/Empty/'
val_path_2   = './data/validation/Occupied/'

os.makedirs(train_path_1, exist_ok=True)
os.makedirs(val_path_1, exist_ok=True)
os.makedirs(train_path_2, exist_ok=True)
os.makedirs(val_path_2, exist_ok=True)

In [ ]:
xml_files = []
jpg_files = []
data_path = '/content/PKLot/PKLot/'

for root, dirs, files in os.walk(data_path):
  for my_file in files:
    if my_file.endswith('.jpg') and os.path.exists(os.path.join(root,my_file[:-4]+'.xml')):
      jpg_files.append(os.path.join(root, my_file))
      xml_files.append(os.path.join(root, my_file[:-4]+'.xml'))

xml_files.sort()
jpg_files.sort()

len(xml_files), len(jpg_files)

In [ ]:
class_ids = ['Occupied', 'Empty']
class_mapping = dict(zip(range(len(class_ids)), class_ids))
class_mapping

In [ ]:
def extract_parking_spaces(image_path, xml_path, save_dir='cropped_images', display_results=False, return_contours=False):

  for img_p, xml_p in zip(image_path, xml_path):
      # Load the image
      img = cv2.imread(img_p)
      img_contours = img.copy()

      # Load the XML file
      tree = ET.parse(xml_p)
      root = tree.getroot()

      # Define a list to store the cropped images
      cropped_images = []
      contours = []

      os.makedirs(save_dir, exist_ok=True)

      # Iterate over the spaces in the XML file
      for space in root.iter('space'):
          # Extract the rotated rectangle and contour from the XML file
          rotated_rect = space.find('rotatedRect')
          contour = space.find('contour')
          center = rotated_rect.find('center')
          size = rotated_rect.find('size')
          angle = rotated_rect.find('angle')

          # Extract the values from the XML elements
          cx, cy = int(center.attrib['x']), int(center.attrib['y'])
          w, h = int(size.attrib['w']), int(size.attrib['h'])
          angle_deg = float(angle.attrib['d'])

          # Convert the angle to radians
          # angle_rad = angle_deg * (3.14159 / 180)

          # Extract the points from the contour
          pts = []
          for point in contour.iter('point'):
              x, y = int(point.attrib['x']), int(point.attrib['y'])
              pts.append((x, y))

          # Create a mask for the polygon
          mask = np.zeros(img.shape[:2], dtype=np.uint8)
          cv2.fillPoly(mask, [np.array(pts)], (255, 255, 255))

          # Rotate the mask by the angle
          M = cv2.getRotationMatrix2D((cx, cy), angle_deg, 1)
          rotated_mask = cv2.warpAffine(mask, M, (img.shape[1], img.shape[0]))

          # Crop the region from the image
          cropped = cv2.bitwise_and(img, img, mask=rotated_mask)

          # Find the bounding box of the contour
          x, y, w, h = cv2.boundingRect(np.array(pts))
          # Crop the region of interest from the cropped image
          roi = img[y:y+h, x:x+w]

          # Resize the region of interest to 100x100
          resized_roi = cv2.resize(roi, (100, 100))

          # Append the cropped image to the list
          cropped_images.append(resized_roi)
          colors = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))

          # Draw the contour on the image
          cv2.drawContours(img_contours, [np.array(pts)], 0, colors, 2)

          contours.append(np.array(pts))

      if display_results:
          plt.grid(linewidth=0)
          plt.imshow(img_contours)
          plt.show()

      # Convert the list of cropped images to a numpy array
      cropped_images_array = np.array(cropped_images)

      contours = np.array(contours)

      if return_contours:
              return cropped_images_array, contours

  return cropped_images_array

In [ ]:
res = extract_parking_spaces([jpg_files[9000]], [xml_files[9000]], display_results=True)
res.shape

In [ ]:
def extract_parking_spaces(jpg_files, json_file, display=True, return_contours=False, roi_size=20):

    # Load the image
    img = cv2.imread(jpg_files)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img_contours = img.copy()
    # Define a list to store the cropped images
    cropped_images = []
    contours = []

    with open(json_file) as f:
      json_dic = json.load(f)

    full_mask = np.zeros(img.shape[:2], dtype=np.uint8)
    for id, shape in enumerate(json_dic['shapes']):
      points = shape['points']
      pts = []
      for p in points:
        pts.append((int(p[0]), int(p[1])))

      mask = np.zeros(img.shape[:2], dtype=np.uint8)
      cv2.fillPoly(mask, [np.array(pts)], (255, 255, 255))
      cv2.fillPoly(full_mask, [np.array(pts)], (255, 255, 255))
      # Crop the region from the image
      cropped = cv2.bitwise_and(img, img, mask=mask)

      # Find the bounding box of the contour
      x, y, w, h = cv2.boundingRect(np.array(pts))
      # Crop the region of interest from the cropped image
      roi = img[y:y+h, x:x+w]

      # Resize the region of interest to 20x20
      resized_roi = cv2.resize(roi, (roi_size, roi_size))

      # Append the cropped image to the list
      cropped_images.append(resized_roi)
      colors = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))

    # Draw the contour on the image
      cv2.drawContours(img_contours, [np.array(pts)], 0, colors, 2)

      contours.append(np.array(pts))

      img_contours = cv2.putText(img_contours, str(id), pts[2], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
      full_mask = cv2.putText(full_mask, str(id), pts[2], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)

    if display:
      plt.imshow(full_mask)
      plt.show()

      plt.grid(linewidth=0)
      plt.imshow(img_contours)
      plt.show()

    cropped_images_array = np.array(cropped_images)

    if return_contours:
          return cropped_images_array, contours
    return cropped_images_array

In [ ]:
jpg_files = '/content/drive/MyDrive/DATA/2012-12-12_10_00_05.jpg'
json_files = '/content/drive/MyDrive/DATA/2012-12-12_10_00_05.json'

s = extract_parking_spaces(jpg_files, json_files)
s.shape

In [ ]:
jpg_files = '/content/drive/MyDrive/DATA/2013-03-09_13_10_08.jpg'
json_files = '/content/drive/MyDrive/DATA/2013-03-09_13_10_08.json'

s = extract_parking_spaces(jpg_files, json_files)
s.shape

In [ ]:
jpg_files = '/content/drive/MyDrive/DATA/2012-10-25_08_23_19.jpg'
json_files = '/content/drive/MyDrive/DATA/2012-10-25_08_23_19.json'

s = extract_parking_spaces(jpg_files, json_files)
s.shape

In [ ]:
parking1 = '/content/PKLot/PKLot/PUCPR/'
parking2 = '/content/PKLot/PKLot/UFPR04/'
parking3 = '/content/PKLot/PKLot/UFPR05/'

mask1 = '/content/drive/MyDrive/DATA/2012-10-25_08_23_19.json'
mask2 = '/content/drive/MyDrive/DATA/2012-12-12_10_00_05.json'
mask3 = '/content/drive/MyDrive/DATA/2013-03-09_13_10_08.json'

In [ ]:
model = pickle.load(open('./drive/MyDrive/DATA/car_spot_model.p', 'rb'))

In [ ]:
def make_predictions(image_path, json_path, classifier=None, display=True):
    lots, contours = extract_parking_spaces(jpg_files, json_files, display=False, return_contours=True)
    image = skimage.io.imread(jpg_files)

    for i, lot in enumerate(lots):
        img = np.asarray(lot.flatten())
        img = np.expand_dims(img, 0)
        pred = model.predict(img)
        # print(pred)
        color = (0, 255, 0) if pred[0] == 1 else (0, 0, 255)
        cv2.drawContours(image, [contours[i]], 0, color, 2)

    if display:
        plt.grid(linewidth=1)
        plt.imshow(image)
        print(image.shape)
        plt.show()

In [ ]:
jpg_files = '/content/drive/MyDrive/DATA/2012-10-25_08_23_19.jpg'
json_files = '/content/drive/MyDrive/DATA/2012-10-25_08_23_19.json'

make_predictions(jpg_files, json_files, classifier=model, display=True)

In [ ]:
jpg_files = '/content/drive/MyDrive/DATA/2013-03-09_13_10_08.jpg'
json_files = '/content/drive/MyDrive/DATA/2013-03-09_13_10_08.json'

make_predictions(jpg_files, json_files, classifier=model, display=True)

In [ ]:
jpg_files = '/content/drive/MyDrive/DATA/2012-12-12_10_00_05.jpg'
json_files = '/content/drive/MyDrive/DATA/2012-12-12_10_00_05.json'

make_predictions(jpg_files, json_files, classifier=model, display=True)

# Parking Spot Detector 2


###Notice:

Since the camera that records the video, moves some inches throughout the whole video, the mask we created, won't be on the exact parking spots so the model won't work that great but you can get a new video and try the script with it in the future.

In [ ]:
import os
import cv2
import numpy as np
from sklearn import metrics
from sklearn import ensemble
from sklearn import linear_model
from sklearn import model_selection

Step 1: get the video from youtube and take it's frames

In [ ]:
# https://www.youtube.com/watch?v=yojapmOkIfg
v1 = './yt5s.io-Busy Parking Lot - Aerial Time-Lapse-(1080p).mp4'

os.makedirs('./frames/', exist_ok=True)

cap = cv2.VideoCapture(v1)

i = 0
while cap.isOpened():
    ret, frame = cap.read()
    i += 1
    if ret:
        cv2.imwrite(f'./frames/{i}.jpg', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

Step2: read each frame and get the labels based on the mask you created from a single frame

In [ ]:
def extract_parking_spaces(jpg_files, json_file, display=True, return_contours=True, return_labels=True, roi_size=100):

    # Load the image
    if type(jpg_files) == str:
        img = cv2.imread(jpg_files)
    else:
        img = jpg_files
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img_contours = img.copy()
    # Define a list to store the cropped images
    cropped_images = []
    contours = []
    labels = []
    with open(json_file) as f:
      json_dic = json.load(f)

    full_mask = np.zeros(img.shape[:2], dtype=np.uint8)
    for id, shape in enumerate(json_dic['shapes']):
      labels.append(int(shape['label']))
      points = shape['points']
      pts = []
      for p in points:
        pts.append((int(p[0]), int(p[1])))

      mask = np.zeros(img.shape[:2], dtype=np.uint8)
      cv2.fillPoly(mask, [np.array(pts)], (255, 255, 255))
      cv2.fillPoly(full_mask, [np.array(pts)], (255, 255, 255))
      # Crop the region from the image
      cropped = cv2.bitwise_and(img, img, mask=mask)

      # Find the bounding box of the contour
      x, y, w, h = cv2.boundingRect(np.array(pts))
      # Crop the region of interest from the cropped image
      roi = img[y:y+h, x:x+w]
      # Resize the region of interest to 20x20
      resized_roi = cv2.resize(roi, (roi_size, roi_size))
      # Append the cropped image to the list
      cropped_images.append(resized_roi)
      colors = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))

    # Draw the contour on the image
      xmin, ymin = np.array(pts)[0]
      xmax, ymax = np.array(pts)[1]
      w = xmax - xmin
      h = ymax - ymin
      Coordinates = [np.array([(xmin, ymin), (xmin+w, ymin), (xmax, ymax), (xmin, ymin+h)])]
#       cv2.drawContours(img_contours, [(xmin, ymin), (xmin+w, ymin), (xmax, ymax), (xmin, ymin+h)], 0, colors, 2)
      cv2.drawContours(img_contours, Coordinates, -1, colors, 2)
      cv2.drawContours(full_mask, Coordinates, -1, colors, 2)

      contours.append(np.array(Coordinates))

    img_contours = cv2.putText(img_contours, str(id), pts[0], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
    full_mask = cv2.putText(full_mask, str(id), pts[0], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)

    if display:
      plt.imshow(full_mask)
      plt.show()

      plt.grid(linewidth=0)
      plt.imshow(img_contours)
      plt.show()

    cropped_images_array = np.array(cropped_images)

    if return_contours:
        if return_labels:
          return cropped_images_array, contours, labels
        else:
          return cropped_images_array, contours
    return cropped_images_array

In [ ]:
jpg_file = './1285.jpg'
json_file = './1285.json'

cropped_images_array, contours, labels = extract_parking_spaces(jpg_file, json_file)

Step3: get some frames with spots and their labels as train/test data


In [ ]:
cropped_images_array, contours, labels = [], [], []
for i in os.listdir('./New folder/'):
    num, types = i.split('.')
    if types == 'jpg':
        jpg_file = f'./drive/MyDrive/DATA/New_folder/{num}.jpg'
        json_file = f'./drive/MyDrive/DATA/New_folder/{num}.json'
        c_img, c, l = extract_parking_spaces(jpg_file, json_file, display=False)
        cropped_images_array.extend(c_img)
        contours.extend(c)
        labels.extend(l)

np.array(contours).shape, np.array(cropped_images_array).shape, np.array(labels).shape

In [ ]:
np.unique(labels, return_counts=True)

Step 4: Bias dataset

Since The dataset is not biased, we only have to limit the dataset to create a balanced dataset


In [ ]:
counter = 0
X = []
Y = []
for img, label in zip(cropped_images_array, labels):
    if label == 0 and counter < 600:
        counter += 1
        X.append(img.flatten()/255)
        Y.append(label)
    if label == 1:
        X.append(img.flatten()/255)
        Y.append(label)

#     X.append(img.flatten()/255)
#     Y.append(label)

train_x, test_x, train_y, test_y = model_selection.train_test_split(X, Y, random_state=32, test_size=0.1)

Train the model

In [ ]:
model = linear_model.SGDClassifier(max_iter=1000, tol=1e-4, loss='log')
model.fit(train_x, train_y)

In [ ]:
pred = model.predict(X)
metrics.accuracy_score(pred, Y)

Test the model on some frames

In [ ]:
def make_predictions(image_path, json_path, model=None, display=True, name='1'):
    lots, contours = extract_parking_spaces(image_path, json_path, display=False, return_contours=True, return_labels=False)
    image = cv2.imread(image_path).copy()

    for i, lot in enumerate(lots):
        img = np.asarray(lot.flatten())
        img = np.expand_dims(img, 0)
        pred = model.predict(img)
        color = (0, 255, 0) if pred[0] == 1 else (255, 0, 0)
        cv2.drawContours(image, [contours[i]], 0, color, 2)

    if display:
        plt.grid(linewidth=1)
        plt.imsave(f'./{name}.jpg',image)
        plt.imshow(image)
        plt.show()

In [ ]:
jpg_file = './4.jpg'
json_file = './1285.json'

make_predictions(jpg_file, json_file, model=model, display=True, name='4a')

In [ ]:
jpg_file = './5.jpg'
json_file = './1285.json'

make_predictions(jpg_file, json_file, model=model, display=True, name='5a')

Test the model on the whole video

In [ ]:
# reading the input
cap = cv2.VideoCapture(v1)
output = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'MPEG'), 30, (520, 720))
frame_nmr = 0
step = 30

while(True):
    ret, frame = cap.read()
    if ret and np.sum(frame) > 0:
        lots, contours = extract_parking_spaces(frame, json_file, display=False, return_contours=True, return_labels=False)
        total = len(lots)
        count = 0
        for i, lot in enumerate(lots):
            img = np.asarray(lot.flatten())
            img = np.expand_dims(img, 0)
            pred = model.predict(img)
            if pred[0] == 1:
                color = (0, 255, 0)
                count += 1
            else:
                color = (0, 0, 255)

            cv2.drawContours(frame, [contours[i]], 0, color, 2)
            output.write(frame)
        cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
        cv2.putText(frame, f'Available spots: {count} / {total}', (100, 60),
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.namedWindow('output', cv2.WINDOW_NORMAL)
        cv2.imshow("output", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        frame_nmr += 1
cv2.destroyAllWindows()
output.release()
cap.release()